In [1]:
import torch

In [17]:
B = 6
K = 3
D = 10

_mixture = torch.randn(B,1,D)
identity_mat = torch.eye(D,D)[None,None]
EPS = 1e-6

Ck = torch.zeros(B,K,D,D)

for b in range(B):
    for i in range(K):
        _Ck = torch.randn(D,D)
        Ck[b,i,:,:] = _Ck @ _Ck.T
        
cov_est_targets = Ck
_est_filters = torch.linalg.solve(
    cov_est_targets.sum(dim=1, keepdim=True).transpose(-2, -1),
    cov_est_targets.transpose(-2, -1)).transpose(-2, -1)
print(cov_est_targets.shape)
print(_est_filters.shape)

torch.Size([6, 3, 10, 10])
torch.Size([6, 3, 10, 10])


In [21]:
mdot = lambda A, B : torch.einsum('...ij,...jk->...ik', A, B)
            
# Covariance of the estimate
Ck = cov_est_targets
# aka Wiener Filter
Wk = _est_filters # = Ck @ inv(Cx)
# Posterior mean
muk = torch.einsum(
     '...ij,...j->...i', Wk, _mixture)
WkCk = mdot(Wk, Ck)
# Posterio covs
pCk = Ck - WkCk + identity_mat * EPS
# Covariance of the mix 
Cx = torch.sum(Ck, dim=1).unsqueeze(1)

# Inverses
Ck_inv = torch.linalg.solve(cov_est_targets, identity_mat)
pCk_inv = Ck_inv + torch.linalg.solve((Cx - Ck), identity_mat)

# Log det
_logdet_Ck = torch.linalg.slogdet(Ck)[-1]
_logdet_pCk = torch.linalg.slogdet(pCk_inv)[-1]

# mu_T @ pCk_inv @ mu
pCkinv_mu = torch.einsum('bkij,bkj->bki', pCk_inv, muk)
mut_pCkinv_mu = torch.einsum('bkj,bkj->bk', muk, pCkinv_mu)

# trace
_tr = torch.diagonal(WkCk, dim1=-2, dim2=-1).sum(dim=-1)

# Loss
loss_kl = torch.sum(0.5 * _tr + _logdet_Ck +  _logdet_pCk + mut_pCkinv_mu, dim=-1)